In [50]:
import swtoolkit as swtk
import win32com.client
import os

def check_file_exists(file_path):
    """Checks if a file exists. Returns True if it does, False if it doesn't."""
    if os.path.isfile(file_path):
        return 
    else:
        open(file_path, "w").close

def get_design_table_from_model_as_list(sw_model):
    """Returns the design table from a model. Returns None if no design table exists."""
    design_table = sw_model.GetDesignTable

    if design_table is None:
        return None
    else:
        dt = []
        nTotRow = design_table.GetTotalRowCount
        nTotCol = design_table.GetTotalColumnCount
        nTotRow = nTotRow + 1
        nTotCol = nTotCol + 1


        design_table.Attach

        # Generate Header Row
        header_row = []
        header_row.append("Config_Name")

        for c_index in range(1, nTotCol):
            header_row.append(design_table.GetHeaderText(c_index-1))

        dt.append(header_row)

        # Populate List of table row data
        for r_index in range(1, nTotRow):
            row_data = []
            for c_index in range(nTotCol):
                row_data.append(design_table.GetEntryValue(r_index, c_index))
            dt.append(row_data)    
        
        design_table.Detach

        return dt



def write_design_table_to_csv(sw_model, csv_file_loc=r'G:\My Drive\Google Drive - Work\TestMacros'):
    """Writes the design table to a csv file with the same name as the model. Creates new CSV if doesnt exist, completely overwrites if it does."""
    # get design table
    model_name = sw_model.GetTitle
    # design_table = sw_model.GetDesignTable
    # nTotRow = design_table.GetTotalRowCount
    # nTotCol = design_table.GetTotalColumnCount
    # nTotRow = nTotRow + 1
    # nTotCol = nTotCol + 1


    # design_table.Attach
    
    
    # # Generate Header Row
    # header_row = "Config_Name,"

    # for c_index in range(1, nTotCol):
    #     header_row = header_row + design_table.GetHeaderText(c_index-1)
    #     if c_index < nTotCol - 1:
    #         header_row = header_row + ","

    # print(f"Header Row: {header_row}\n\n")

    # # Populate List of table row data
    # table_data = []
    # for r_index in range(1, nTotRow):
    #     row_data = ""
    #     for c_index in range(nTotCol):
    #         print(f"{design_table.GetEntryText(r_index, c_index)}")
    #         row_data = row_data + design_table.GetEntryText(r_index, c_index)
    #         if c_index < nTotCol - 1:
    #             row_data = row_data + ","
    #     table_data.append(row_data)

    # print(f"Table Data:")
    # print(header_row)
    # for row in table_data:
    #     print(f"   {row}")


    # design_table.Detach

    # Get design table as list
    design_table = get_design_table_from_model_as_list(sw_model)
    

    nTotRow = len(design_table)
    nTotCol = len(design_table[0])
    print(f"Design Table Rows: {nTotRow}")
    print(f"Design Table Columns: {nTotCol}")

    csv_file_name = csv_file_loc + "\\" + model_name + ".csv"
    print(f"CSV File Name: {csv_file_name}\n\n")

    for row in design_table:
        print(row)

    with open(csv_file_name, 'w') as csv_file:
        for r_index in range(nTotRow):
            for c_index in range(nTotCol):
                csv_file.write(f"{str(design_table[r_index][c_index])}")
                if c_index < nTotCol - 1:
                    csv_file.write(",")
            csv_file.write("\n")


    return



# sw = win32com.client.Dispatch("SldWorks.Application")
sw = swtk.SolidWorks()

# get active document
model = sw._active_doc()
# model = sw.ActiveDoc
write_design_table_to_csv(model)
# print(f"\n\n{model.GetTitle}\n\n")

# # print(type(model))

# # get design table
# design_table = model.GetDesignTable

# # print(type(design_table))

# bRet = design_table.Attach

# print(bRet)

# nTotRow = design_table.GetTotalRowCount
# nTotCol = design_table.GetTotalColumnCount

# print(f"Number of Rows:    {nTotRow}")
# print(f"Number of Columns: {nTotCol}\n\n")

# nTotRow = nTotRow + 1
# nTotCol = nTotCol + 1

# for r_index in range(1, nTotRow):
#     for c_index in range(nTotCol):
#         print(f"Row: {r_index} Column: {c_index} | Column: {design_table.GetHeaderText(c_index-1)}   Value: {design_table.GetEntryValue(r_index, c_index)}")

# print(f"\nNow using Entry Text:\n")

# for r_index in range(1, nTotRow):
#     for c_index in range(nTotCol):
#         print(f"Row: {r_index} Column: {c_index} | Column: {design_table.GetHeaderText(c_index-1)}   Value: {design_table.GetEntryText(r_index, c_index)}")



# bRet = design_table.Detach

sw_model = sw.get_model()
sw_model.extension.rebuild(1)
print(f"\n\nDone.\n\n")



Design Table Rows: 3
Design Table Columns: 8
CSV File Name: G:\My Drive\Google Drive - Work\TestMacros\Template_Panel.csv


['Config_Name', '$DESCRIPTION', '$COLOR', 'Horizontal_Dim@Panel_Sketch', 'Vertical_Dim@Panel_Sketch', 'Horizontal_Centre_Offset@Panel_Sketch', 'Vertical_Centre_Offset@Panel_Sketch', 'Panel_Thickness_Dim@Panel_Thickness']
['Default', 'Default', 15651274.0, 4.3885071, 4.0, 0.0, 0.0, 0.7500000000000001]
['Test1', 'Test1', 15651274.0, 5.0, 3.0000000000000004, 0.1, 0.1, 1.0]


Done.


